In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"fake_news.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna()

In [5]:
X = df.drop('label',axis=1)

In [6]:
y = df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
df.shape

(18285, 5)

In [9]:
import tensorflow as tf

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
voc_size=5000

# OneHot Representation

In [12]:
messages = X.copy()

In [13]:
messages.reset_index(inplace=True)

In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sahas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review =[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [18]:
onehot_repr = [one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2765, 4682, 55, 718, 1976, 4986, 1641, 2012, 2658, 618],
 [2636, 1972, 1009, 3455, 2563, 1517, 4870],
 [2227, 185, 668, 4116],
 [523, 4699, 1567, 2138, 1244, 4816],
 [1773, 2563, 2960, 1440, 2372, 4346, 2563, 3926, 694, 364],
 [4735,
  1144,
  193,
  4518,
  3106,
  3358,
  2202,
  3687,
  2031,
  1353,
  2550,
  2841,
  2772,
  2620,
  4870],
 [4882, 4564, 1661, 4033, 1323, 4183, 3385, 1230, 542, 1215, 4976],
 [1882, 2081, 1598, 1019, 2514, 4770, 3358, 2738, 542, 1215, 4976],
 [2191, 612, 3380, 3529, 4942, 511, 2582, 4833, 3358, 4855],
 [4594, 4015, 4466, 131, 381, 2653, 2822, 567],
 [3122, 4374, 4294, 2922, 3149, 1674, 3423, 4044, 3473, 4570, 72],
 [2138, 312, 1976, 511, 3358, 2514],
 [2270, 1944, 497, 4510, 4544, 1818, 182, 1675, 532],
 [557, 3297, 2089, 1652, 1451, 3221, 4948, 542, 1215, 4976],
 [1339, 1650, 2848, 1891, 1169, 542, 1215, 4976],
 [2755, 1226, 2053, 1536, 4372, 2210, 3136, 3870, 3681, 1288],
 [3815, 2986, 1972],
 [797, 2837, 1201, 176, 3358, 3553, 832, 4870],
 [397,

onehot_repr basically takes the word and voc_size as paramaters and marks the word 1 in an index of voc_size 

# Embedding  Representation

As all the sentence lengths are different so before making it pass through the embedding layer
i have to make the sentences of equal length.So I will do this by pad_sequencing

In [19]:
sent_length=20
embedded_docs = pad_sequences(onehot_repr,padding ='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2012 2658  618]
 [   0    0    0 ... 2563 1517 4870]
 [   0    0    0 ...  185  668 4116]
 ...
 [   0    0    0 ...  542 1215 4976]
 [   0    0    0 ... 2840 4956  913]
 [   0    0    0 ... 2598 2343 4473]]


In [20]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2765,
       4682,   55,  718, 1976, 4986, 1641, 2012, 2658,  618])

# Creating Model

what all vector_features it will undertake is very difficult to tell so i am defining the number of vector_features(random number) here we are taking 40 

In [21]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = sent_length))
model.add(LSTM(100))  #this implies that 1 LSTM layer contains 100 Neurons
model.add(Dense(1,activation ='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer ='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [23]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

# Model Training

In [25]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 18ms/step - loss: 0.3324 - accuracy: 0.8410 - val_loss: 0.1942 - val_accuracy: 0.9190
Epoch 2/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1429 - accuracy: 0.9466 - val_loss: 0.2005 - val_accuracy: 0.9225
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1006 - accuracy: 0.9642 - val_loss: 0.2031 - val_accuracy: 0.9229
Epoch 4/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0754 - accuracy: 0.9743 - val_loss: 0.2437 - val_accuracy: 0.9085
Epoch 5/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0596 - accuracy: 0.9791 - val_loss: 0.2548 - val_accuracy: 0.9092
Epoch 6/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0453 - accuracy: 0.9852 - val_loss: 0.3450 - val_accuracy: 0.9155
Epoch 7/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0353 - accuracy: 0.9878 - val_loss: 0.3819 - val_accuracy: 0.9082

# Performance Metrics and Accuracy

In [31]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(y_test,y_pred)

array([[3156,  263],
       [ 282, 2334]], dtype=int64)

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9096934548467275